In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 3000
ATTACK_NAME = 'DoS'

In [2]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [3]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_hulk_goldeneye_samples_closed_port.csv')
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,2,220.363764,50,296,67.600983,4569.892957,232140,262,189.967267,18,63.418444,258,164.564356,16,85.622521,0.000000,0.000000,198.410256,0,0,0,39.669597,35.896508,1.000031,0.027877,0.070697
1,2,196.643612,50,207,20.506782,420.528127,337874,173,163.698643,18,17.048190,82,41.666667,16,23.478122,0.000000,0.000000,167.929423,0,0,0,40.280836,51.687110,0.117685,0.019356,0.011854
2,1,127.500000,92,163,35.500000,1260.250000,638,58,58.000000,58,0.000000,129,129.000000,129,0.000000,0.000000,0.000000,63.800000,0,0,0,33.391080,0.658859,14.915988,1.590051,4.257473
3,0,115.000000,115,115,0.000000,0.000000,0,0,0.000000,0,0.000000,81,81.000000,81,0.000000,0.000000,0.000000,0.000000,0,0,0,36.147329,0.940595,4.095852,1.095374,0.532049
4,1,98.452381,93,103,4.552354,20.723923,838,40,39.904762,39,0.293544,49,49.000000,49,0.000000,0.000000,0.000000,41.900000,0,0,0,38.551803,1.089443,2.105557,0.940288,0.933142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,1,670.961538,64,1292,557.940373,311297.460059,6510,1208,342.631579,30,435.891248,1258,806.424242,51,550.035593,0.000000,0.000000,723.333333,0,0,0,4.517126,11.511744,2.497524,0.088571,0.381031
231,1,105.875000,66,197,48.485662,2350.859375,396,163,99.000000,32,52.368884,83,44.750000,32,22.083648,67.000000,12.750000,198.000000,0,8,0,24.115742,0.331734,23.837876,3.445106,8.325512
232,1,300.340426,86,1172,336.539099,113258.564962,7686,1078,334.173913,32,376.451008,1118,162.166667,32,267.411616,301.826087,129.666667,0.000000,2,46,0,0.983398,47.793469,0.117976,0.021378,0.033776
233,1,92.000000,92,92,0.000000,0.000000,696,58,58.000000,58,0.000000,0,0.000000,0,0.000000,0.000000,0.000000,116.000000,0,0,0,15.058889,0.796872,14.949139,1.368990,4.294515


In [4]:
# get all the relevant attack rows from the attack sample dataset
dosSamples = dosSamples[dosSamples['SYN Flag Count'] >= 50]

In [5]:
# print some general information about the attack samples
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples = dosSamples.reset_index(drop=True)
dosSamples

Dataset Shape: (8, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,74.0,74,74,0.0,0.0,20240,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,3373.333333,506,0,0,1.260528,401.419060,0.814980,0.002496,0.036519
1,1,74.0,74,74,0.0,0.0,20280,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,2897.142857,507,0,0,2.241037,226.234538,0.956779,0.004429,0.056727
2,1,74.0,74,74,0.0,0.0,20560,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1468.571429,514,0,0,3.208441,160.202415,1.023966,0.006254,0.069266
3,1,74.0,74,74,0.0,0.0,20480,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1706.666667,512,0,0,2.302322,222.384193,0.895812,0.004506,0.054606
4,1,74.0,74,74,0.0,0.0,36680,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,58.407643,917,0,0,18.027555,50.866576,8.869790,0.019681,0.319971
5,1,74.0,74,74,0.0,0.0,24640,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,77.728707,616,0,0,7.291802,84.478433,3.481875,0.011857,0.144467
6,1,74.0,74,74,0.0,0.0,24360,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,78.834951,609,0,0,9.749890,62.462243,6.866742,0.016036,0.278521
7,1,74.0,74,74,0.0,0.0,12320,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1540.000000,308,0,0,2.122772,145.093304,0.767681,0.006915,0.045732


In [6]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [7]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (float(dosSamples[col].min() * 0.9), float(dosSamples[col].max() * 1.1)) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (66.60000000000001, 81.4),
 'Packet Length Min': (66.60000000000001, 81.4),
 'Packet Length Max': (66.60000000000001, 81.4),
 'Total Length of Fwd Packet': (11088.0, 40348.0),
 'Fwd Packet Length Max': (36.0, 44.0),
 'Fwd Packet Length Mean': (36.0, 44.0),
 'Fwd Packet Length Min': (36.0, 44.0),
 'Subflow Fwd Bytes': (52.56687898089172, 3710.666666666667),
 'SYN Flag Count': (277.2, 1008.7),
 'Flow Duration': (1.1344752788543702, 19.830310487747195),
 'Packets Per Second': (45.77991860293527, 441.56096596998174),
 'IAT Max': (0.6909127950668336, 9.756768822669983),
 'IAT Mean': (0.00224648570070171, 0.02164881057614319),
 'IAT Std': (0.032867191553272196, 0.35196773982329765)}

In [8]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (66.60000000000001, 81.4),
 'Packet Length Min': (66.60000000000001, 81.4),
 'Packet Length Max': (66.60000000000001, 81.4),
 'Total Length of Fwd Packet': (11088.0, 40348.0),
 'Fwd Packet Length Max': (36.0, 44.0),
 'Fwd Packet Length Mean': (36.0, 44.0),
 'Fwd Packet Length Min': (36.0, 44.0),
 'Subflow Fwd Bytes': (52.56687898089172, 3710.666666666667),
 'SYN Flag Count': (277, 1008),
 'Flow Duration': (1.1344752788543702, 19.830310487747195),
 'Packets Per Second': (45.77991860293527, 441.56096596998174),
 'IAT Max': (0.6909127950668336, 9.756768822669983),
 'IAT Mean': (0.00224648570070171, 0.02164881057614319),
 'IAT Std': (0.032867191553272196, 0.35196773982329765)}

In [9]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

In [11]:
same_value = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']
val = np.random.randint(MinMaxDict[same_value[0]][0], MinMaxDict[same_value[0]][1]*1.1, NUM_OF_ROWS)

for col in same_value:
    dosDataset[col] = val

In [12]:
same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']
val2 = np.random.randint(MinMaxDict[same_value2[0]][0], MinMaxDict[same_value2[0]][1]*1.25, NUM_OF_ROWS)

for col in same_value2:
    dosDataset[col] = val2

In [13]:
dosDataset['Flow Duration'] = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.95, MinMaxDict['Flow Duration'][1]*1.05, NUM_OF_ROWS)
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset['Subflow Fwd Bytes'] = np.full(shape=NUM_OF_ROWS, fill_value=0, dtype=int)
dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0]*0.9, MinMaxDict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)

In [14]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'Total Length of Fwd Packet']
independent_col = dosSamples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[first_correlation[1]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)
    
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        delta = random.uniform(factor * 0.05, factor * 0.25) # select a dantom
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

('Total Length of Fwd Packet', np.float64(40.00000000000001))


In [15]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

('Packets Per Second', np.float64(8.979156689418554))
('IAT Max', np.float64(0.5248448565577644))
('IAT Mean', np.float64(0.001311749361677395))
('IAT Std', np.float64(0.020313533395492172))


np.float64(561.125)

In [16]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr*0.05, durationToPacketsCorr * 0.125) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.1, factor * 0.25)
                updatedFactor = factor + random.choices([-1, 1], weights=[1, 2], k=1)[0] * delta  
            elif col == 'IAT Max':
                delta = random.uniform(factor * 0.1, factor * 0.25)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.25, factor * 0.5) # select a delta
                updatedFactor = factor + random.choices([-1, 1], weights=[1, 3], k=1)[0] * delta
            dosDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [17]:
rand_values = np.random.uniform(MinMaxDict['Subflow Fwd Bytes'][0], MinMaxDict['Subflow Fwd Bytes'][1], NUM_OF_ROWS)
usual_values = np.random.uniform(1105.763, 3642.146, NUM_OF_ROWS)

# Choose values randomly (25% from rand_values, 75% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.25, usual_values, rand_values) #! later in the second sample there will be rows with subflow fwd bytes = 0

dosDataset['Subflow Fwd Bytes'] = chosen_values

In [18]:
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,74.0,74,74,0.0,0.0,20240,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,3373.333333,506,0,0,1.260528,401.419060,0.814980,0.002496,0.036519
1,1,74.0,74,74,0.0,0.0,20280,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,2897.142857,507,0,0,2.241037,226.234538,0.956779,0.004429,0.056727
2,1,74.0,74,74,0.0,0.0,20560,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1468.571429,514,0,0,3.208441,160.202415,1.023966,0.006254,0.069266
3,1,74.0,74,74,0.0,0.0,20480,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1706.666667,512,0,0,2.302322,222.384193,0.895812,0.004506,0.054606
4,1,74.0,74,74,0.0,0.0,36680,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,58.407643,917,0,0,18.027555,50.866576,8.869790,0.019681,0.319971
5,1,74.0,74,74,0.0,0.0,24640,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,77.728707,616,0,0,7.291802,84.478433,3.481875,0.011857,0.144467
6,1,74.0,74,74,0.0,0.0,24360,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,78.834951,609,0,0,9.749890,62.462243,6.866742,0.016036,0.278521
7,1,74.0,74,74,0.0,0.0,12320,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,1540.000000,308,0,0,2.122772,145.093304,0.767681,0.006915,0.045732


In [19]:
x = dosDataset[dosDataset['Flow Duration'] > 2.2]
x[x['Flow Duration'] < 2.3][:20]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
50,1,81,81,81,0,0,9074.0,42,42,42,0,0,0,0,0,0,0,2373.604624,249,0,0,2.248753,231.041099,1.366550,0.004075,0.037480
143,1,80,80,80,0,0,27212.0,41,41,41,0,0,0,0,0,0,0,189.632808,889,0,0,2.241482,278.871512,1.041753,0.001789,0.054510
248,1,71,71,71,0,0,21020.0,41,41,41,0,0,0,0,0,0,0,2276.243748,692,0,0,2.283644,223.119722,1.436620,0.002148,0.035267
339,1,82,82,82,0,0,13090.0,40,40,40,0,0,0,0,0,0,0,3039.287302,399,0,0,2.246034,223.149865,1.357887,0.004056,0.055753
344,1,85,85,85,0,0,11973.0,46,46,46,0,0,0,0,0,0,0,1271.276238,334,0,0,2.207199,233.030121,1.282246,0.002049,0.035031
412,1,77,77,77,0,0,22037.0,42,42,42,0,0,0,0,0,0,0,1713.176539,622,0,0,2.245198,236.429966,0.924553,0.004102,0.052690
585,1,78,78,78,0,0,15404.0,51,51,51,0,0,0,0,0,0,0,1786.050218,340,0,0,2.204909,226.979849,0.974235,0.001954,0.049816
662,1,76,76,76,0,0,19066.0,44,44,44,0,0,0,0,0,0,0,1885.062338,546,0,0,2.275495,271.823220,0.925948,0.004458,0.051881
717,1,79,79,79,0,0,49956.0,53,53,53,0,0,0,0,0,0,0,2874.357159,1036,0,0,2.235051,220.388045,1.430519,0.001875,0.037098
1557,1,82,82,82,0,0,26635.0,47,47,47,0,0,0,0,0,0,0,2749.985316,567,0,0,2.214393,274.482231,0.999819,0.003808,0.054628


In [20]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,8.0,8.0,8.0,8.0,8.0,8.0,8.000000,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.000000,8.000000,8.0,8.0,8.000000,8.000000,8.000000,8.000000,8.000000
mean,1.0,74.0,74.0,74.0,0.0,0.0,22445.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1400.085698,561.125000,0.0,0.0,5.775543,169.142595,2.959703,0.009022,0.125726
std,0.0,0.0,0.0,0.0,0.0,0.0,6872.302588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1284.388466,171.807565,0.0,0.0,5.770372,115.414002,3.204683,0.006176,0.112626
min,1.0,74.0,74.0,74.0,0.0,0.0,12320.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.407643,308.000000,0.0,0.0,1.260528,50.866576,0.767681,0.002496,0.036519
25%,1.0,74.0,74.0,74.0,0.0,0.0,20270.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.558390,506.750000,0.0,0.0,2.211471,78.974385,0.875604,0.004486,0.052388
50%,1.0,74.0,74.0,74.0,0.0,0.0,20520.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1504.285714,513.000000,0.0,0.0,2.755381,152.647860,0.990372,0.006584,0.062997
75%,1.0,74.0,74.0,74.0,0.0,0.0,24430.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2004.285714,610.750000,0.0,0.0,7.906324,223.346779,4.328092,0.012901,0.177980
max,1.0,74.0,74.0,74.0,0.0,0.0,36680.000000,40.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3373.333333,917.000000,0.0,0.0,18.027555,401.419060,8.869790,0.019681,0.319971


In [21]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,3000.0,3000.000000,3000.000000,3000.000000,3000.0,3000.0,3000.000000,3000.000000,3000.000000,3000.000000,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.000000,3000.000000,3000.0,3000.0,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,1.0,76.758000,76.758000,76.758000,0.0,0.0,26875.438667,44.946667,44.946667,44.946667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2241.410451,671.797000,0.0,0.0,10.880740,84.400487,5.719523,0.016971,0.233134
std,0.0,6.712311,6.712311,6.712311,0.0,0.0,10954.954034,5.461724,5.461724,5.461724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,868.679908,247.888266,0.0,0.0,5.682022,84.182461,3.225325,0.010406,0.128786
min,1.0,66.000000,66.000000,66.000000,0.0,0.0,7725.000000,36.000000,36.000000,36.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.426247,249.000000,0.0,0.0,1.078788,23.752061,0.444770,0.000757,0.017559
25%,1.0,71.000000,71.000000,71.000000,0.0,0.0,17731.500000,40.000000,40.000000,40.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1573.783361,454.000000,0.0,0.0,5.889773,35.267009,3.032853,0.008528,0.123262
50%,1.0,77.000000,77.000000,77.000000,0.0,0.0,26301.000000,45.000000,45.000000,45.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2268.080713,674.000000,0.0,0.0,10.951942,51.475877,5.453036,0.014960,0.226698
75%,1.0,83.000000,83.000000,83.000000,0.0,0.0,34623.500000,50.000000,50.000000,50.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2983.783110,884.250000,0.0,0.0,15.670083,95.220695,8.030588,0.025320,0.327205
max,1.0,88.000000,88.000000,88.000000,0.0,0.0,55138.000000,54.000000,54.000000,54.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3710.360714,1107.000000,0.0,0.0,20.818236,557.584532,13.582078,0.040697,0.526703


---

In [22]:
dosDataset['Label'] = ATTACK_NAME

In [23]:
dosDataset.shape

(3000, 27)

In [ ]:
# save the dataset
# dosDataset.to_csv('dos_hulk_goldeneye_closed_port_dataset_new.csv', index=False)